In [1]:
import dask
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster, progress
import us
import os

In [9]:
try:
    cluster
except NameError:
    # create LocalCluster w/ dask
    cluster = LocalCluster()
    client = Client(cluster)
    # then look at diagnostic dash here http://localhost:8787/status or http://127.0.0.1:8787/status
    client
    # # to turn off
    # client.shutdown()

In [27]:
client

Client Scheduler: tcp://127.0.0.1:57625 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [31]:
# combine files
@dask.delayed
def read_files(csv_dir):
    combined_df = []
    for item in os.listdir(csv_dir):
        if item.endswith(".csv"):
            file_name = csv_dir + item
            print(file_name)
            df = dd.read_csv(file_name,assume_missing=True)
            combined_df.append(df)
    combined_df = dd.concat(combined_df)
    return combined_df

In [23]:
future_nyc_citibike = client.compute(read_files("../data/citibike-tripdata-nyc/"))
progress(future_nyc_citibike)

VBox()

In [25]:
future_nyc_citibike

<Future: finished, type: dask.DataFrame, key: read_files-99c24996-71db-4493-af4e-f9ef47e192df>

In [26]:
nyc_df = future_nyc_citibike.result()
nyc_df.head(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362.0,2017-09-01 00:00:17,2017-09-01 00:06:19,3331.0,Riverside Dr & W 104 St,40.801343,-73.971146,3328.0,W 100 St & Manhattan Ave,40.795000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,188.0,2017-09-01 00:00:21,2017-09-01 00:03:30,3101.0,N 12 St & Bedford Ave,40.720798,-73.954847,3100.0,Nassau Ave & Newell St,40.724813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,305.0,2017-09-01 00:00:25,2017-09-01 00:05:30,3140.0,1 Ave & E 78 St,40.771404,-73.953517,3141.0,1 Ave & E 68 St,40.765005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,223.0,2017-09-01 00:00:52,2017-09-01 00:04:36,236.0,St Marks Pl & 2 Ave,40.728419,-73.987140,473.0,Rivington St & Chrystie St,40.721101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,758.0,2017-09-01 00:01:01,2017-09-01 00:13:40,3427.0,Lafayette St & Jersey St,40.724305,-73.996010,3431.0,E 35 St & 3 Ave,40.746524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2089.0,2017-09-01 00:01:20,2017-09-01 00:36:09,3016.0,Kent Ave & N 7 St,40.720368,-73.961651,3358.0,Garfield Pl & 8 Ave,40.671198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,121.0,2017-09-01 00:01:22,2017-09-01 00:03:23,3357.0,W 106 St & Amsterdam Ave,40.800836,-73.966449,3343.0,W 107 St & Columbus Ave,40.799757,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,408.0,2017-09-01 00:01:39,2017-09-01 00:08:27,470.0,W 20 St & 8 Ave,40.743453,-74.000040,388.0,W 26 St & 10 Ave,40.749718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,485.0,2017-09-01 00:01:41,2017-09-01 00:09:46,513.0,W 56 St & 10 Ave,40.768254,-73.988639,529.0,W 42 St & 8 Ave,40.757570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,220.0,2017-09-01 00:01:40,2017-09-01 00:05:21,405.0,Washington St & Gansevoort St,40.739323,-74.008119,358.0,Christopher St & Greenwich St,40.732916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
future_jc_citibike = client.compute(read_files("../data/citibike-tripdata-jc/"))
progress(future_jc_citibike)


VBox()

In [30]:
%%time 
# checkout missing data situation 
len(nyc_df['starttime'])

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+------------------+---------+----------+
| Column           | Found   | Expected |
+------------------+---------+----------+
| end station id   | float64 | int64    |
| start station id | float64 | int64    |
+------------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'end station id': 'float64',
       'start station id': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [ ]:
# total trips per month
# most popular start station
# most popular routes (start + end station)
# average trip length by month? by day of week?
%%time
nyc_df.groupby


In [ ]:
states = [i.abbr for i in us.STATES]
nyc_residents = dd.read_csv("../data/covid_19_data.csv")
final_df = []


for state in states:
    #     maybe you need path as well?
    path = os.getcwd()
    filename = str(path+"/"+state+".csv")
    current_state_df = dd.read_csv(f"{filename}", header=[])
    merge_df = df.merge(left=nyc_residents, right=current_state_df, how="inner", on=['pid', 'addrid'])
    final_df.append(merge_df)
final_df = pd.concat(final_df)


In [ ]:
"https://stackoverflow.com/questions/65619777/display-progress-on-dask-computesomething-call"